## Libraries

In [1]:
import ast
import base64
import contextlib
import datetime
import io
from io import StringIO
# import json
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import streamlit as st
import textwrap
from PIL import Image
from vertexai.generative_models import Part
from vertexai.language_models import ChatModel
from vertexai.generative_models import GenerativeModel
from vertexai.preview import generative_models
from vertexai.language_models import InputOutputTextPair

## Datasets

In [2]:
# Import the data
df = pd.read_csv("Ro_IBRO_input.csv")

In [3]:
cltv = pd.read_csv("cltv.csv")
cltv['MONTH_ID'] = pd.to_datetime(cltv['MONTH_ID'], format='%d/%m/%Y')

## Data pre-processing functions

In [4]:
# Function to extract year and month details from the data set
def extract_year_month(
    input_df,
    date_col,
    date_format='%Y%m',
):
    """
    This function helps to extract month and year from the date column.
    input:
    input_df (dataframe): pandas dataframe
    date_col (str): name of the date column
    date_format (str): the format to parse the date, default='%Y%m'
    output:
    input_df[date, year_col, month_col, quarter_col]
    """
    # Convert the REPORTING_DATE column to datetime
    input_df['date'] = pd.to_datetime(input_df[date_col], format=date_format)
    # Extract the year, month and quarter from the date column
    input_df['year'] = input_df['date'].dt.year
    input_df['month'] = input_df['date'].dt.month
    input_df['quarter'] = input_df['date'].dt.quarter
    input_df['year_month'] = pd.PeriodIndex(input_df.date, freq='M')
    input_df['year_quarter'] = pd.PeriodIndex(input_df.date, freq='Q')
    input_df.drop([date_col], axis=1, inplace=True)
    return input_df

In [5]:
# Function to generate the schema details
def get_schema_info(input_df):
    """
    This function creates a basic schema for the given dataset.
    It creates a dictionaries with the unique values for categorical columns
    and percentiles of numeric columns
    input:
    input_df (dataframe): pandas dataframe
    output:
    numeric_dict (dict),  object_dict {dict}
    numeric_dict: dictionary with numeric data type information
    object_dict: dictionary with object data type informatio
    """
    numeric_dict = {}
    object_dict ={}
    for col in input_df:
        if input_df[col].dtype == 'O':
            object_dict[col] = input_df[col].unique()
        else:
            numeric_dict[col] = input_df.agg({col:["min","max"]})
    return (numeric_dict, object_dict)

## Other functions

In [6]:
@contextlib.contextmanager
def stdoutIO(stdout=None):
    old = sys.stdout
    if stdout is None:
        stdout = StringIO()
    sys.stdout = stdout
    yield stdout
    sys.stdout = old

In [7]:
def is_valid_python(code_string):
    try:
        ast.parse(code_string)
    except SyntaxError:
        return False
    return True

## Data prep

In [8]:
revenue_df = extract_year_month(df, "accounting_rpt_month", "%Y-%m-%d")
revenue_df.head(2)

,TRANSACTION_TYPE,Product,ProductSub,Segment,Channel,County,Region,Subs_Count,Total_S15_Revenue,PREV_Total_S15_Revenue,...,Inbundle_S15_Revenue,PREV_Inbundle_S15_Revenue,Outbundle_S15_Revenue,PREV_Outbundle_S15_Revenue,date,year,month,quarter,year_month,year_quarter
0,OUTFLOW,M2M,M2M,SMB,INDIRECT HUNTING,Bucuresti,Sud,2681,0.0,149.0,...,0.0,4.0,0.0,145.0,2024-01-01,2024,1,1,2024-01,2024Q1
1,BASE,M2M,M2M,MLE,INDIRECT SALES,Bucuresti,Sud,4806,2747.0,2747.0,...,2584.0,2584.0,163.0,163.0,2024-01-01,2024,1,1,2024-01,2024Q1


In [9]:
cltv_df = extract_year_month(cltv, "MONTH_ID", "%Y-%m-%d")
cltv_df.head(2)

,EBU_SEGMENT,ACQ_CHANNEL,PRODUCT_TYPE,CLT,CLTV_USER,AVG_REV_MONTH,CLTV,INFLOW_BASE,date,year,month,quarter,year_month,year_quarter
0,any,BSA,M2M,40.017427,72.94422,1.822811,7732.087322,106,2024-03-01,2024,3,1,2024-03,2024Q1
1,SMB,BSA,M2M,40.017427,72.94422,1.822811,7732.087322,106,2024-03-01,2024,3,1,2024-03,2024Q1


In [10]:
# Extract max dates from sales and costs data
revenue_df_end_date = max(revenue_df['date'])
cltv_df_end_date = max(cltv_df['date'])

# create data set information for LLM context
revenue_df_info = revenue_df.dtypes
cltv_df_info = cltv_df.dtypes

# create a meta data for revenue_df and cltv
num_dict, obj_dict= get_schema_info(revenue_df)
revenue_df_desc = dict(list(obj_dict.items())+list(num_dict.items()))

num_dict, obj_dict= get_schema_info(cltv_df)
cltv_df_desc = dict(list(obj_dict.items())+list(num_dict.items()))

##########################################
##     Set up the context for LLM        #
##########################################

In [11]:
 revenue_df_schema_info = """This dataset appears to contain information about transactions, likely for a company operating in Romania. Here's a breakdown:

Column name description Information:

connectivity revenue refers to total_s15_revenue.
The column names with "prev" as prefix refers to outflow activity.

- Transaction Type: "INFLOW, BASE, RETENTION, OUTFLOW" transactions are present, indicating this dataset focuses on revenue-generating activities.

- Products: The dataset covers various products, including fixed lines, mobile voice, mobile data, M2M services, and potentially others.

- Segments: Customer segments are categorized as SMB (Small and Medium Businesses), MLE, Strategic, Public, Wholesale, and "Other".

- Channels: The data includes information on the sales channels used, including Direct Sales, Indirect Sales, BSA (Business Sales Associate), Digital, and several others.

- Geography: County and Region level information is available.

- Revenue and Subscribers: Data points include total revenue, ARPU (Average Revenue Per User), Inbundle/Outbundle revenue, and subscriber counts(volume), giving insight into revenue generation and customer behavior.

- Time: Data is collected over a period of time, with monthly, quarterly, and yearly breakdowns.
""" + str(revenue_df_info)

cltv_df_schema_info = """This dataset appears to contain information about customer lifetime and customer lifetime value. Also dataset contain inflow count for combined segment, channel and product. Here's a breakdown:

Column name description Information:

- PRODUCT_TYPE: The dataset covers various products, including fixed lines, mobile voice, mobile data, M2M services, and potentially others.

- EBU SEGMENT: Customer segments are categorized as SMB (Small and Medium Businesses), MLE, Strategic, Public, Wholesale, and "Other".

- ACQ CHANNEL: The data includes information on the sales channels used, including Direct Sales, Indirect Sales, BSA (Business Sales Associate), Digital, and several others.

- CLT and CLTV: Data points include CLT and CLTV giving insight into Customer lifetime and customer lifetime value.

- INFLOW_BASE: INFLOW_BASE(volume) is count of inflow 

- Time: Data is collected over a period of time, with monthly, quarterly, and yearly breakdowns.
""" + str(cltv_df_info)

In [12]:
context_to_llm = f"""You are a top level python programmer.
Your job is to convert the natural language questions related to the datasets
into a python snippet.
If the question is not related to below given datasets schemas,
then generate a standard answer as
"Sorry, I am trained to answer questions related to the importance data and customer lifetime value(CLTV) for Romania."
Datasets information:
revenue_df schema: {revenue_df_schema_info},
revenue_df data column-wise descriptions: {revenue_df_desc},
cltv schema: {cltv_df_schema_info},
cltv data column-wise descriptions: {cltv_df_desc}
If the question is related to the given datasets schemas,
then convert the natural language questions into a python snippet.
Only consider information available in the data schema for code generation.
Here are some general instructions.
- Combine segment, channel and product to calculate CLT and CLTV at a cohort level.
- Revenue, ARPU and subscriber count(volume) are the key KPI's.
- Consider all the KPI's mentioned above to analyse the performance, in case it is not given in the question.
- YTD means year till date.
- FY is financial year which starts from April to March next year.
- CLT means Customer lifetime
- CLTV means Customer lifetime value
- churn means outflow
- If date is not mention in question then take max date
- Revenue is in euros.
- Current date for revenue_df is {revenue_df_end_date}.
- Current date for cltv_df is {cltv_df_end_date}.
- Do not add any textual explaination while generating a python snippet.
- Do not modify the dataframes.
Only python code should be an output of every question.
Follow the instructions carefully before generating a response.
previous_answer:""
"""

In [13]:
business_context = f"""
You are a business analyst working with two datasets: `revenue_df` and `cltv_df`. 
- `revenue_df` contains information about customer transactions.
- `cltv_df` includes data on customer lifetime, inflow count, and customer lifetime value across various channels, segments, and products.

Below are the schema and column details:
- `revenue_df` schema: {revenue_df_info}
- `cltv_df` schema: {cltv_df_info}
- Column descriptions for `revenue_df`: {revenue_df_desc}
- Column descriptions for `cltv_df`: {cltv_df_desc}

Using the provided table information, the user's question, and the Python output (either text or plot table), frame a professional statement for a business reader. Ensure the statement:
- Is based solely on the given data.
- Includes numerical details.
- Is brief and to the point.
- Accurately describes the plot by referring to the `plot_table` and the date ranges.
- Arrange numerical information in descending order when nothing menthion about order.
- Once you start with any product channel or anything first complete about it and then talk about another one.

Take a deep breath and proceed.
"""

In [14]:
# Instruction text to extract plot description
entity_extraction_instr = f"""You are an expert plot dataframe name extractor.
Refer the last python snippet to extract the dataframe object.
Extract the dataframe object names which are being plotted from the given python code.
Output should be the dataframe object names only,
returned as a string object in this format "plotted_object: plot object names"

#################
Example 1:
python_code: import matplotlib.pyplot as plt
import pandas as pd
import io
import base64

# Group the sales_df by date and calculate the sum of sales
sales_df_monthly = sales_df.groupby('date')['SALE'].sum().reset_index();

# Group the costs_df by date and calculate the sum of costs
costs_df_monthly = costs_df.groupby('date')['COST'].sum().reset_index();

# Create a figure with two subplots
fig, axs = plt.subplots(2, 1, sharex=True);

# Plot the sales on the first subplot
axs[0].plot(sales_df_monthly['date'], sales_df_monthly['SALE'], color='blue', label='Sales');
axs[0].set_ylabel('Sales');
axs[0].legend();

# Plot the costs on the second subplot
axs[1].plot(costs_df_monthly['date'], costs_df_monthly['COST'], color='red', label='Costs');
axs[1].set_ylabel('Costs');
axs[1].legend();
plt.clf();
plt.xlabel('Date');
plt.ylabel('Amount');
plt.title('Sales and Cost Trend');
plt.legend();

# Save the plot to a buffer
buf = io.BytesIO();
plt.savefig(buf, format='png');

# Convert the buffer to a binary string
plot_binary = base64.b64encode(buf.getbuffer()).decode('ascii');

plotted_object: sales_df_monthly, costs_df_monthly
 
#################
Example 2:
python_code:
# Filter the costs_df to only include data for the year 2022
costs_df_2022 = costs_df[costs_df['year'] == 2022]

# Group the data by month and calculate the total cost
costs_df_2022_monthly = costs_df_2022.groupby('year_month')['COST'].sum()

plt.clf();
# Create a plot of the monthly cost trend
costs_df_2022_monthly.plot()
plt.xlabel('Month')
plt.ylabel('Cost')
plt.title('Cost Trend for 2022')

plotted_object: costs_df_2022_monthly

#################
Example 3:
python_code:
# Filter the sales_df to only include data for the year 2023
sales_df_2023 = sales_df[sales_df['year'] == 2023];

# Group the data by month and calculate the total sales
sales_by_month_2023 = sales_df_2023.groupby('month')['SALE'].sum().reset_index();

plt.clf();
# Create a line plot of the sales trend for 2023
plt.plot(sales_by_month_2023['month'], sales_by_month_2023['SALE']);
plt.xlabel('Month');
plt.ylabel('Sales');
plt.title('Sales Trend for 2023');

plotted_object: sales_by_month_2023

#################
"""

######################################################
##    Few shot examples for within context            #
######################################################

In [15]:
# prepare training example list for the llm
train_example_list=[
    InputOutputTextPair(
        input_text="""For inflow, which is my top sales channel.""",
        output_text="""# Filter the data for inflow transaction type
inflow_rev_df = revenue_df[revenue_df['TRANSACTION_TYPE'] == 'INFLOW']
inflow_cltv_df = cltv_df[cltv_df['TRANSACTION_TYPE'] == 'INFLOW']

# Group the data by channel and calculate sum of subscriber count
Total_channel_subs_count = inflow_rev_df.groupby('Channel')['Subs_Count'].sum()

# Group the data by channel and calculate mean of subscriber count
channel_subs_count = inflow_rev_df.groupby('Channel')['Subs_Count'].mean()

# Group the data by channel and calculate sum of total/connectivity revenue
channel_revenue = inflow_rev_df.groupby('Channel')['Total_S15_Revenue'].sum()

# Filter the cltv data for EBU_SEGMENT and PRODUCT_TYPE only for any and ACQ_CHANNEL without any
cltv_without_any = cltv_df[(cltv_df['ACQ_CHANNEL']!='any') & (cltv_df['PRODUCT_TYPE']=='any') & (cltv_df['EBU_SEGMENT']=='any')]

# Group the data by channel and calculate mean of CLTV
channel_cltv = cltv_without_any.groupby('ACQ_CHANNEL')['CLTV'].mean()

# Group the data by channel and calculate mean of CLT
channel_clt = cltv_without_any.groupby('ACQ_CHANNEL')['CLT'].mean()

# Calculate ARPU for each channel
channel_arpu = channel_revenue/Total_channel_subs_count

# Get the channel with the highest subscriber count
top_channel_name_for_subs_count = channel_subs_count.idxmax()
top_channel_subs_count_value = channel_subs_count.max()

# Get the channel with the highest total revenue
top_channel_name_for_revenue = channel_revenue.idxmax()
top_channel_revenue_value = channel_revenue.max()

# Get the channel with the highest ARPU
top_channel_name_for_arpu = channel_arpu.idxmax()
top_channel_arpu_value = channel_arpu.max()

# Get the channel with the highest CLTV
top_channel_name_for_cltv = channel_cltv.idxmax()
top_channel_cltv_value = channel_cltv.max()

# Get the channel with the highest CLT
top_channel_name_for_clt = channel_clt.idxmax()
top_channel_clt_value = channel_clt.max()

# Print the top product information for volume, total revenue, ARPU, CLTV and CLT
print(f"top_channel in terms of volume {top_channel_name_for_subs_count} with value {top_channel_subs_count_value}")
print(f"top_channel in terms of total revenue {top_channel_name_for_revenue} with value {top_channel_revenue_value}")
print(f"top_channel in terms of ARPU {top_channel_name_for_arpu} with value {top_channel_arpu_value}")
print(f"top_channel in terms of CLTV {top_channel_name_for_cltv} with value {top_channel_cltv_value}")
print(f"top_channel in terms of CLT {top_channel_name_for_clt} with value {top_channel_clt_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""Which sales channel contributes the most in the SMB segment""",
        output_text="""# Filter the dataset for the SMB segment
smb_df = revenue_df[revenue_df['Segment'] == 'SMB']

# Group the data by channel and calculate sum of subscriber count
Total_channel_subs_count = smb_df.groupby('Channel')['Subs_Count'].sum()

# Group the data by channel and calculate mean of subscriber count
channel_subs_count = smb_df.groupby('Channel')['Subs_Count'].mean()

# Group the data by channel and calculate sum of total/connectivity revenue
channel_revenue = smb_df.groupby('Channel')['Total_S15_Revenue'].sum()

# Filter the cltv data for PRODUCT_TYPE and EBU_SEGMENT only for any and ACQ_CHANNEL without any
cltv_without_any = cltv_df[(cltv_df['ACQ_CHANNEL']!='any') & (cltv_df['PRODUCT_TYPE']=='any') & (cltv_df['EBU_SEGMENT']=='any')]

# Group the data by channel and calculate mean of CLTV
channel_cltv = cltv_without_any.groupby('ACQ_CHANNEL')['CLTV'].mean()

# Group the data by channel and calculate mean of CLT
channel_clt = cltv_without_any.groupby('ACQ_CHANNEL')['CLT'].mean()

# Calculate ARPU for each channel
channel_arpu = channel_revenue/Total_channel_subs_count

# Get the channel with the highest subscriber count
top_channel_name_for_subs_count = channel_subs_count.idxmax()
top_channel_subs_count_value = channel_subs_count.max()

# Get the channel with the highest total revenue
top_channel_name_for_revenue = channel_revenue.idxmax()
top_channel_revenue_value = channel_revenue.max()

# Get the channel with the highest ARPU
top_channel_name_for_arpu = channel_arpu.idxmax()
top_channel_arpu_value = channel_arpu.max()

# Get the channel with the highest CLTV
top_channel_name_for_cltv = channel_cltv.idxmax()
top_channel_cltv_value = channel_cltv.max()

# Get the channel with the highest CLT
top_channel_name_for_clt = channel_clt.idxmax()
top_channel_clt_value = channel_clt.max()

# Print the top product information for volume, total revenue, ARPU, CLTV and CLT
print(f"top_channel in terms of volume {top_channel_name_for_subs_count} with value {top_channel_subs_count_value}")
print(f"top_channel in terms of total revenue {top_channel_name_for_revenue} with value {top_channel_revenue_value}")
print(f"top_channel in terms of ARPU {top_channel_name_for_arpu} with value {top_channel_arpu_value}")
print(f"top_channel in terms of CLTV {top_channel_name_for_cltv} with value {top_channel_cltv_value}")
print(f"top_channel in terms of CLT {top_channel_name_for_clt} with value {top_channel_clt_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""Revenue generated by each product in last month?""",
        output_text="""# find the last month from data
last_month = revenue_df['year_month'].max()-1

# Filter the data for last month
last_month_df = revenue_df[revenue_df['year_month']==last_month]

# Group the data by product and calculate sum of total revenue
product_revenue = last_month_df.groupby('Product')['Total_S15_Revenue'].sum()

# Print the product with the highest revenue
print(f"Product with highest revenue {product_revenue.idxmax()} with value {product_revenue.max()}")

# Print the revenue generated by each product
for product, revenue in product_revenue.items():
    print(f"{product}: {revenue}")
"""
    ),
    InputOutputTextPair(
        input_text="""Which channel has worst ARPU?""",
        output_text="""# Calculate ARPU for each channel
channel_arpu = revenue_df.groupby('Channel')['ARPU'].mean()

# Get the channel with the lowest ARPU
worst_channel_arpu = channel_arpu.idxmin()

# Get the channel value with the lowest ARPU
worst_channel_arpu_value = channel_arpu.min()

# Print the worst channel with value
print(f"Channel with lowest ARPU: {worst_channel_arpu} with {worst_channel_arpu_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""Which is the best performing Segment according to CLT?""",
        output_text="""# find the current month from cltv dataframe
current_month = cltv_df['year_month'].max()

# Filter the data for current month
current_month_cltv = cltv_df[cltv_df['year_month']==current_month]

# Filter the cltv data for ACQ_CHANNEL and PRODUCT_TYPE only for any and EBU_SEGMENT without any
clt_with_any = current_month_cltv[(current_month_cltv['ACQ_CHANNEL']=='any') & (current_month_cltv['PRODUCT_TYPE']=='any') & (current_month_cltv['EBU_SEGMENT']!='any')]

# Get the Segment with the highest CLTV
top_Segment_cltv = cltv_with_any['CLTV'].idxmax()

# Get the Segment value with the highest CLTV
top_Segment_cltv_value = cltv_with_any['CLTV'].max()

# Print the highest Segment with value
print(f"Segment with highest CLTV: {top_Segment_cltv} with {top_Segment_cltv_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""Which product has the maximum CLTV for the month of 202403?""",
        output_text="""# Filter the data for the month of 202403
cltv_202403 = cltv_df[cltv_df['year_month']=='2024-03']

# Filter the cltv data for ACQ_CHANNEL and EBU_SEGMENT only for any and PRODUCT_TYPE without any
cltv_with_any = cltv_202403[(cltv_202403['ACQ_CHANNEL']=='any') & (cltv_202403['PRODUCT_TYPE']!='any') & (cltv_202403['EBU_SEGMENT']=='any')]

# Get the Segment with the highest CLTV
top_product_cltv = cltv_with_any['CLTV'].idxmax()

# Get the Segment value with the highest CLTV
top_product_cltv_value = cltv_with_any['CLTV'].max()

# Print the highest CLTV with value
print(f"Product with highest CLTV: {top_product_cltv} with {top_product_cltv_value} for month 202403")
"""
    ),
    InputOutputTextPair(
        input_text="""Which is the best performing channel as per clt for march 2024""",
        output_text="""# Filter the data for the month of 202403
clt_df_202403 = cltv_df[cltv_df['year_month']=='2024-03']

# Filter the clt data for EBU_SEGMENT and PRODUCT_TYPE only for any and ACQ_CHANNEL without any
clt_with_any = clt_df_202403[(clt_df_202403['EBU_SEGMENT']=='any') & (clt_df_202403['PRODUCT_TYPE']=='any') & (clt_df_202403['ACQ_CHANNEL']!='any')]

# Sort by Revenue in descending order and get top 3
top3_sales_channel_clt = clt_with_any.sort_values(by='CLT', ascending=False).head(3)

print("Top 3 Channel by CLT:")
print(top3_sales_channel_clt)
"""
    ),
    InputOutputTextPair(
        input_text="""Which is the best performing channel as per cltv?""",
        output_text="""# find the current month from cltv dataframe
current_month = cltv_df['year_month'].max()

# Filter the cltv data for current month
current_month_cltv = cltv_df[cltv_df['year_month']==current_month]

# Filter the cltv data for EBU_SEGMENT and PRODUCT_TYPE only for any and ACQ_CHANNEL without any
cltv_with_any = current_month_cltv[(current_month_cltv['EBU_SEGMENT']=='any') & (current_month_cltv['PRODUCT_TYPE']=='any') & (current_month_cltv['ACQ_CHANNEL']!='any')]

# Sort by Revenue in descending order and get top 3
top3_sales_channel_cltv = cltv_with_any.sort_values(by='CLTV', ascending=False).head(3)

print("Top 3 Channel by CLTV:")
print(top3_sales_channel_cltv)
"""
    ),
    InputOutputTextPair(
        input_text="""Which is the best performing segment for March-2024?""",
        output_text="""# Filter the data for the month of 202403
revenue_df_202403 = revenue_df[revenue_df['year_month']=='2024-03']

# Group the data by channel and calculate sum of subscriber count
Total_segment_subs_count = revenue_df_202403.groupby('Segment')['Subs_Count'].sum()

# Group the data by Segment and calculate mean of subscriber count
Segment_subs_count = revenue_df_202403.groupby('Segment')['Subs_Count'].mean()

# Group the data by Segment and calculate sum of total/connectivity revenue
Segment_revenue = revenue_df_202403.groupby('Segment')['Total_S15_Revenue'].sum()

# Filter the cltv data for ACQ_CHANNEL and PRODUCT_TYPE only for any and EBU_SEGMENT without any

cltv_df_202403_without_any = cltv_df[(cltv_df['year_month']=='2024-03') & (cltv_df['ACQ_CHANNEL']=='any') & (cltv_df['PRODUCT_TYPE']=='any') & (cltv_df['EBU_SEGMENT']!='any')]

# Group the data by channel and calculate mean of CLTV
channel_cltv = cltv_df_202403_without_any.groupby('ACQ_CHANNEL')['CLTV'].mean()

# Group the data by channel and calculate mean of CLT
channel_clt = cltv_df_202403_without_any.groupby('ACQ_CHANNEL')['CLT'].mean()

# Calculate ARPu for each Segment
Segment_arpu = Segment_revenue/Total_segment_subs_count

# Get the Segment with the highest subscriber count
top_Segment_name_for_subs_count = Segment_subs_count.idxmax()
top_Segment_subs_count_value = Segment_subs_count.max()

# Get the Segment with the highest total revenue
top_Segment_name_for_revenue = Segment_revenue.idxmax()
top_Segment_revenue_value = Segment_revenue.max()

# Get the Segment with the highest arpu
top_Segment_name_for_arpu = Segment_arpu.idxmax()
top_Segment_arpu_value = Segment_arpu.max()

# Get the channel with the highest CLTV
top_Segment_name_for_cltv = channel_cltv.idxmax()
top_Segment_cltv_value = channel_cltv.max()

# Get the channel with the highest CLT
top_Segment_name_for_clt = channel_clt.idxmax()
top_Segment_clt_value = channel_clt.max()

# Print the top Segment information for volume, total revenue and ARPU
print(f"top_Segment in terms of volume {top_Segment_name_for_subs_count} with value {top_Segment_subs_count_value}")
print(f"top_Segment in terms of total revenue {top_Segment_name_for_revenue} with value {top_Segment_revenue_value}")
print(f"top_Segment in terms of ARPU {top_Segment_name_for_arpu} with value {top_Segment_arpu_value}")
print(f"top_channel in terms of CLTV {top_Segment_name_for_cltv} with value {top_Segment_cltv_value}")
print(f"top_channel in terms of CLT {top_Segment_name_for_clt} with value {top_Segment_clt_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""Which sales channel contributes most in the SMB segment with respect rev?""",
        output_text="""# Filter for SMB segment
smb_df = revenue_df[revenue_df['Segment'] == 'SMB']

# find the current month from cltv dataframe
current_month = smb_df['year_month'].max()

# Filter the data for current month
current_month_revenue_df = smb_df[smb_df['year_month']==current_month]

# Group the data by channel and calculate sum of total/connectivity revenue
channel_revenue = current_month_revenue_df.groupby('Channel')['Total_S15_Revenue'].sum().reset_index()

# Sort by Revenue in descending order and get top 3
top3_sales_channel_rev = channel_revenue.sort_values(by='Total_S15_Revenue', ascending=False).head(3)

print("Top 3 Channel by CLTV:")
print(top3_sales_channel_rev)
"""
    ),
    InputOutputTextPair(
        input_text="""What is overall CLTV?""",
        output_text="""# Calculate the overall CLTV Filter the cltv data for ACQ_CHANNEL, PRODUCT_TYPE and EBU_SEGMENT only for any
cltv_only_any = cltv_df[(cltv_df['ACQ_CHANNEL']=='any') & (cltv_df['PRODUCT_TYPE']=='any') & (cltv_df['EBU_SEGMENT']=='any')]
overall_cltv = cltv_only_any.loc[cltv_only_any['CLTV'].idxmax()]

# Print the overall CLTV
print(f"Overall CLTV: {overall_cltv}")
"""
    ),
    InputOutputTextPair(
        input_text="""Tell me the top 3 channels by churn""",
        output_text="""# Filter the data for outflow transaction type
churn_df = revenue_df[revenue_df['TRANSACTION_TYPE'] == 'OUTFLOW']

# Group the data by channel and calculate mean of subscriber count
channel_subs_count = churn_df.groupby('Channel')['Subs_Count'].mean()

# Sort the channel_subs_count in descending order and get the top channels
top3_channel_churn = channel_subs_count.sort_values(ascending=False).head(3)

print("Top 3 channel by churn:")
print(top3_channel_churn)
"""
    ),
    InputOutputTextPair(
        input_text="""Which product contributes the most in the SMB segment""",
        output_text="""# Filter the dataset for the SMB segment
smb_df = revenue_df[revenue_df['Segment'] == 'SMB']

# Group the data by channel and calculate sum of subscriber count
Total_product_subs_count = smb_df.groupby('Channel')['Subs_Count'].sum()

# Group the data by product and calculate mean of subscriber count
product_subs_count = smb_df.groupby('Product')['Subs_Count'].mean()

# Group the data by product and calculate sum of total/connectivity revenue
product_revenue = smb_df.groupby('Product')['Total_S15_Revenue'].sum()

# Filter the cltv data for ACQ_CHANNEL and EBU_SEGMENT only for any and PRODUCT_TYPE without any
cltv_without_any = cltv_df[(cltv_df['ACQ_CHANNEL']=='any') & (cltv_df['PRODUCT_TYPE']!='any') & (cltv_df['EBU_SEGMENT']=='any')]

# Group the data by product and calculate mean of CLTV
product_cltv = cltv_without_any.groupby('PRODUCT_TYPE')['CLTV'].mean()

# Group the data by product and calculate mean of CLT
product_clt = cltv_without_any.groupby('PRODUCT_TYPE')['CLT'].mean()

# Calculate ARPU for each product
product_arpu = product_revenue/Total_product_subs_count

# Get the product with the highest subscriber count
top_product_name_for_subs_count = product_subs_count.idxmax()
top_product_subs_count_value = product_subs_count.max()

# Get the product with the highest total revenue
top_product_name_for_revenue = product_revenue.idxmax()
top_product_revenue_value = product_revenue.max()

# Get the product with the highest ARPU
top_product_name_for_arpu = product_arpu.idxmax()
top_product_arpu_value = product_arpu.max()

# Get the product with the highest CLTV
top_product_name_for_cltv = product_cltv.idxmax()
top_product_cltv_value = product_cltv.max()

# Get the product with the highest CLT
top_product_name_for_clt = product_clt.idxmax()
top_product_clt_value = product_clt.max()

# Print the top product information for volume, total revenue, ARPU, CLTV and CLT
print(f"top_product in terms of volume {top_product_name_for_subs_count} with value {top_product_subs_count_value}")
print(f"top_product in terms of total revenue {top_product_name_for_revenue} with value {top_product_revenue_value}")
print(f"top_product in terms of ARPU {top_product_name_for_arpu} with value {top_product_arpu_value}")
print(f"top_product in terms of CLTV {top_product_name_for_cltv} with value {top_product_cltv_value}")
print(f"top_product in terms of CLT {top_product_name_for_clt} with value {top_product_clt_value}")
"""
    ),
    InputOutputTextPair(
        input_text="""What is the reason for drop in revenue?""",
        output_text="""# find the current month from data
current_month = revenue_df['year_month'].max()

# find the last month from data
last_month = revenue_df['year_month'].max()-1

# Filter the data for outflow transaction type and current month
current_month_churn_df = revenue_df[(revenue_df['TRANSACTION_TYPE'] == 'OUTFLOW') & (revenue_df['year_month']==current_month)]

# Filter the data for outflow transaction type and last month
last_month_churn_df = revenue_df[(revenue_df['TRANSACTION_TYPE'] == 'OUTFLOW') & (revenue_df['year_month']==last_month)]

# Calculate sum of total/connectivity revenue for current month
current_month_churn_rev = current_month_churn_df['Total_S15_Revenue'].sum()

# Calculate sum of total/connectivity revenue for last month
last_month_churn_rev = last_month_churn_df['Total_S15_Revenue'].sum()

# Calculate mean of subscriber count for current month
current_month_churn_subs_count = current_month_churn_df['Subs_Count'].mean()

# Calculate mean of subscriber count for last month
last_month_churn_subs_count = last_month_churn_df['Subs_Count'].mean()

# Filter the data for inflow transaction type and current month
current_month_inflow_df = revenue_df[(revenue_df['TRANSACTION_TYPE'] == 'INFLOW') & (revenue_df['year_month']==current_month)]

# Filter the data for inflow transaction type and last month
last_month_infow_df = revenue_df[(revenue_df['TRANSACTION_TYPE'] == 'INFLOW') & (revenue_df['year_month']==last_month)]


# Calculate sum of total/connectivity revenue for current month
current_month_rev_inflow = current_month_inflow_df['Total_S15_Revenue'].sum()

# Calculate sum of total/connectivity revenue for last month
last_month_rev_inflow = last_month_infow_df['Total_S15_Revenue'].sum()

# Calculate mean of subscriber count for current month
current_month_subs_count_inflow = current_month_inflow_df['Subs_Count'].mean()

# Calculate mean of subscriber count for last month
last_month_subs_count_inflow = last_month_infow_df['Subs_Count'].mean()

# Check if the revenue for OUTFLOW transactions has increased in the latest month compared to the previous month
if last_month_churn_rev < current_month_churn_rev:
    print(f'churn revenue increases in current month by {current_month_churn_rev - last_month_churn_rev}')
    # Check if the subscriber count for OUTFLOW transactions has increased in the latest month compared to the previous month
    if last_month_churn_subs_count < current_month_churn_subs_count:
        print(f'churn increases in current month by {current_month_churn_subs_count - last_month_churn_subs_count} subscriber count')
    else:
        print('Not found any reason for this month could you mention specfic month')
else:
    print('Not found any reason for this month could you mention specfic month')

# Check if the revenue for INFLOW transactions has decreased in the latest month compared to the previous month
if current_month_rev_inflow < last_month_rev_inflow:
    print(f'inflow revenue decrease in current month by {last_month_rev_inflow - current_month_rev_inflow}')
    # Check if the subscriber count for INFLOW transactions has decreased in the latest month compared to the previous month
    if current_month_subs_count_inflow < last_month_subs_count_inflow:
        print(f'inflow decrease in current month by {last_month_subs_count_inflow - current_month_subs_count_inflow} subscriber count')
    else:
        print('Not found any reason for this month could you mention specfic month')
else:
    print('Not found any reason for this month could you mention specfic month')
"""
    ),
    InputOutputTextPair(
        input_text="""Which product has been consistently increasing revenue?""",
        output_text="""# Calculate the monthly revenue for each product
product_monthly_revenue = revenue_df.groupby(['Product', 'year_month'])['Total_S15_Revenue'].sum().reset_index()

# Calculate the percentage change in revenue for each product over the previous month
product_monthly_revenue['revenue_change'] = product_monthly_revenue['Total_S15_Revenue'].pct_change()

# Filter the data to only include products with a positive revenue change in the last 3 months
products_with_consistent_growth = product_monthly_revenue[(product_monthly_revenue['year_month'].isin(product_monthly_revenue['year_month'].unique()[-3:])) & (product_monthly_revenue['revenue_change'] > 0)]

# Print the products with consistent revenue growth
print("Products with consistent revenue growth:")
print(products_with_consistent_growth)
"""
    )
    
]

## Import LLM models

In [16]:
# import chat bison model to generate the python code
chat_model = ChatModel.from_pretrained("chat-bison-32k")

# Start the chat model
code_gen_llm = chat_model.start_chat(
    context=context_to_llm, examples = train_example_list
)

In [17]:
# Configuration parameters for chat bison
code_gen_parameters = {
    "max_output_tokens": 1024,
    "temperature": 0.1,
    "top_p": 0.8,
    "top_k": 40
}

In [18]:
# call the gemini model
gemini_model = GenerativeModel("gemini-1.5-flash-001")

# start text gen LLM model
text_gen_llm = gemini_model.start_chat()

In [19]:
# Gemini model configuration
gemini_config = {
    "max_output_tokens": 8092,
    "temperature": 0.1,
    "top_p": 0.95
}

# saftey setting for gemini model
gemini_safety_setting = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [20]:
# call the gemini model
gemini_extrt_model = GenerativeModel("gemini-1.0-pro-002")

In [21]:
ent_extract_config = {
    "max_output_tokens": 256,
    "temperature": 0,
    "top_p": 0.8
}

In [22]:
# Function to extract entity from python code
def extract_entity(
    python_code_instr,
    code_extractor_model,
    entity_extrt_model_config,
    safety_settings
    ):
    """
    python_code_instr (str): string of python code and the instructions

    return: entities extracted from python code
    """
    safety_settings = gemini_safety_setting

    entities = code_extractor_model.generate_content(
      [python_code_instr],
      generation_config = entity_extrt_model_config,
      safety_settings = safety_settings,
      stream=True
    )

    objects = ""
    for entity in entities:
        objects += entity.text

    return objects

In [23]:
plot_desc_model_config = {
    "max_output_tokens": 2048,
    "temperature": 0.1,
    "top_p": 0.7
}

## Start talking to the bot

In [24]:
def question_answer(question):
    # generates the python code
    response = code_gen_llm.send_message(question, **code_gen_parameters)
    response.text
    py_code = response.text
    code_output = py_code.split('```')[1].split('python')[1] if "python" in py_code else py_code
    print(code_output)
    if is_valid_python(code_output):
        # Execute the code output
        exec(code_output)
        # if image not generated in code (text output)
        if code_output.find('plot') == -1:
            with stdoutIO() as s:
                exec(code_output)
            exe_output = s.getvalue()
            text_input=business_context + f"""User input: {question}, 
            python output: {exe_output}"""

            rephrased_answer = text_gen_llm.send_message(
                [text_input],
                generation_config=gemini_config,
                safety_settings=gemini_safety_setting
                )
            return rephrased_answer.text
        else:
            # Diplay the plot on UI
            plt.show()
            # if the code consists an image
            py_code_instr = f"""instructions: {entity_extraction_instr}\n
            python code: {code_output}"""

            # gemini entity extraction model configuration
            ent_config = ent_extract_config
            plot_object_name = extract_entity(
                python_code_instr=py_code_instr,
                code_extractor_model=gemini_extrt_model,
                entity_extrt_model_config=ent_config,
                safety_settings=gemini_safety_setting
            )
            plot_object = eval(
                plot_object_name.split('plotted_object: ')[1]
            )
            # st.text(plot_object)

            context_text = business_context + f"""table: {plot_object} +
            question: {question}"""

            rephrased_answer = text_gen_llm.send_message(
                [context_text],
                generation_config=plot_desc_model_config,
                safety_settings=gemini_safety_setting
                )
            return rephrased_answer.text
    else:
        print("check python code")
        rephrased_answer = "check python code"
        return rephrased_answer

## Questions

In [25]:
# question = "Which channel contributes the most in the SMB segment?"
# question = "Which product contributes the most in the SMB segment?"
# question = "Which are the top 3 contributing segment according to ARPU?"
# question = "What is the contribution of mobile and fixed product in MLE segement?"
# question = "What is sales generated by each product in last month?"
# question = "Revenue generated by each product in current month?"
# question = "Which channel contributes highest in south region?"
# question = "Which channel has the highest average revenue per user for the current month?"
# question = "Which price plan has best mobile ARPU in SMB seggment?"
# question = "Which channel contributes the most?"
# question = "Show it for subscriber count"
# question = "What is the contribution of mobile and fixed product in MLE segement?
# question = "give me bifurcation of mobile voice and mobile data"
# question = "What is the contribution of mobile and fixed product in MLE segement? and give me bifurcation of mobile voice and mobile data"
# question = "Tell me the top 10 channel by churn"

## Follow up questions

In [26]:
# question = "Revenue generated by each product in current month?"
# question = "What about last month?"
# question = "Which is the least contributing channel in current month?"
# question = "Show the contribution of each channel at a region level"

## Combined Questions

In [27]:
# question = "Which channel contributes the most in the SMB segment?"
# question = "Which product contributes the most in the SMB segment?"
# question = "Which are the top 3 contributing segment according to ARPU?"
# question = "What is the contribution of mobile and fixed product in MLE segement?"
# question = "What is sales generated by each product in last month?"
# question = "Revenue generated by each product in current month?"
# question = "Which channel contributes highest in south region?"
# question = "Which channel has the highest average revenue per user for the current month?"
# question = "Which price plan has best mobile ARPU in SMB seggment?"
# question = "Which channel contributes the most?"
# question = "Show it for subscriber count"
# question = "What is the contribution of mobile and fixed product in MLE segement?"
# question = "give me bifurcation of mobile voice and mobile data"
# question = "What is the contribution of mobile and fixed product in MLE segement? and give me bifurcation of mobile voice and mobile data"
# question = "Tell me the top 10 channel by churn"
# question = "Revenue generated by each product in current month?"
# question = "What about last month?"
# question = "Which is the least contributing channel in current month?"
# question = "Show the contribution of each channel at a region level"
# question = "Which are lowest priceplan for Churn?"
# question = "Which are top region for Churn?"
# question = "Which are top 3 lowest channel for Churn?"
# question = "Tell me the top 2 customer product by revenue?"
# question = "Tell me the top channel according base"

## Reasoning Questions
# question = "What is the reason for drop in revenue?"
# question = "What is the reason for drop in inflow?"
question = "What is the product breakup of my inflow for SMB segment?"

# question = "Which product has been consistently increasing revenue?"

In [28]:
# question = "What is the reason for drop in inflow for jan month?"

In [29]:
ans = question_answer(question=question)


# Filter the data for inflow transaction type and SMB segment
inflow_smb_df = revenue_df[(revenue_df['TRANSACTION_TYPE'] == 'INFLOW') & (revenue_df['Segment'] == 'SMB')]

# Group the data by product and calculate sum of subscriber count
product_subs_count = inflow_smb_df.groupby('Product')['Subs_Count'].sum()

# Group the data by product and calculate sum of total/connectivity revenue
product_revenue = inflow_smb_df.groupby('Product')['Total_S15_Revenue'].sum()

# Calculate ARPU for each product
product_arpu = product_revenue/product_subs_count

# Print the product breakup of inflow for SMB segment
print("Product breakup of inflow for SMB segment:")
print(product_subs_count)
print(product_revenue)
print(product_arpu)

Product breakup of inflow for SMB segment:
Product
Fixed Line       3172
M2M              1969
Mobile Data       223
Mobile Voice    21639
Name: Subs_Count, dtype: int64
Product
Fixed Line       10677.0
M2M               1284.0
Mobile Data       1131.0
Mobile Voice    23

In [30]:
ans

'For the SMB segment, Mobile Voice is the leading product in terms of inflow, with 21,639 subscriptions, followed by Fixed Line (3,172), M2M (1,969), and Mobile Data (223). This trend is reflected in the total revenue generated, with Mobile Voice contributing the most at 233,011, followed by Fixed Line (10,677), M2M (1,284), and Mobile Data (1,131).  The average revenue per user (ARPU) for Mobile Voice is the highest at 10.77, followed by Mobile Data (5.07), Fixed Line (3.37), and M2M (0.65). \n'

In [31]:
question

'What is the product breakup of my inflow for SMB segment?'

In [32]:
cltv

,EBU_SEGMENT,ACQ_CHANNEL,PRODUCT_TYPE,CLT,CLTV_USER,AVG_REV_MONTH,CLTV,INFLOW_BASE,date,year,month,quarter,year_month,year_quarter
0,any,BSA,M2M,40.017427,72.944220,1.822811,7732.087322,106,2024-03-01,2024,3,1,2024-03,2024Q1
1,SMB,BSA,M2M,40.017427,72.944220,1.822811,7732.087322,106,2024-03-01,2024,3,1,2024-03,2024Q1
2,any,DSA,M2M,75.641521,139.792090,1.848087,6989.604510,50,2024-03-01,2024,3,1,2024-03,2024Q1
3,SMB,DSA,M2M,51.764783,107.874898,2.083944,5393.744886,50,2024-03-01,2024,3,1,2024-03,2024Q1
4,any,RAM,M2M,20.900579,33.809709,1.617645,135.238837,4,2024-03-01,2024,3,1,2024-03,2024Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,MLE,TAM DIRECT SALES,Mobile Voice,58.475112,913.882253,15.628568,449630.068300,492,2023-03-01,2023,3,1,2023-03,2023Q1
2187,any,INDIRECT FRANCHISES,Mobile Voice,36.332035,504.443625,13.884266,70117.663830,139,2023-03-01,2023,3,1,2023-03,2023Q1
2188,SMB,INDIRECT FRANCHISES,Mobile Voice,36.332035,504.443625,13.884266,70117.663830,139,2023-03-01,2023,3,1,2023-03,2023Q1
2189,Wholesale,any,Mobile Voice,34.037794,244.960520,7.196721,0.000000,0,2023-03-01,2023,3,1,2023-03,2023Q1


## Suggestions

## Rought Work 

In [33]:
# question = "Which channel contributes the most in the SMB segment?"
# question = "Which product contributes the most in the SMB segment?"
# question = "Which are the top 3 contributing segment according to ARPU?"
# question = "What is the contribution of mobile and fixed product in MLE segement?"
# question = "What is sales generated by each product in last month?"
# question = "Revenue generated by each product in current month?"
# question = "Which channel contributes highest in south region?"
# question = "Which channel has the highest average revenue per user for the current month?"
# question = "Which price plan has best mobile ARPU in SMB seggment?"
# question = "Which channel contributes the most?"
# question = "Show it for subscriber count"
# question = "What is the contribution of mobile and fixed product in MLE segement?
# question = "give me bifurcation of mobile voice and mobile data"
# question = "What is the contribution of mobile and fixed product in MLE segement? and give me bifurcation of mobile voice and mobile data"
# question = "Tell me the top 10 channel by churn"

## Follow up questions
# question = "Revenue generated by each product in current month?"
# question = "What about last month?"
# question = "Which is the least contributing channel in current month?"
# question = "Show the contribution of each channel at a region level"

# question = "What is the reason for drop in revenue?" # Not working
# question = "What is the reason for drop in inflow?" # Not working
# question = "Which are lowest priceplan for Churn?"
# question = "Tell me the top 2 customer product by revenue?"
# question = "Which product has been consistently increasing revenue?"
# question = "What is the product breakup of revenue for SMB segment?"

# question = "What is the reason for drop in revenue?"
# question = "Which are top channel for Churn?"
# question = "Which are lowest priceplan for Churn?"
# question = "Which are top region for Churn?"
# question = "Which are top 3 lowest channel for Churn?"

# question = "What is the product breakup of my inflow for SMB segment?"
# question = "What is the reason for drop in inflow"
# question = "Which customer has been consistently increasing revenue?"
# question = "Tell me the top channel according base"
# question = "What is the reason for drop in revenue"
# question = "Which are top channel for Churn"